In [346]:
# Library Import
import os 
import pandas as pd
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

pd.options.mode.use_inf_as_na = True

In [347]:
# Parameters

Host = "https://strategenics.atlassian.net/"
Username = "mike.burns@strategenics.com.au"
Password = "ATATT3xFfGF0wcJfZvitiiWKYW_9UCdV2r1dWJE6O8a7ZxFRQUU7FVnBkdYlxNx9PcEn_tbHM-tdxz4DZedCozWuNUSI46DcZDPMKpXq0QDgiWomXTEkzDjVZfVa4oRjADfcbgebgME5skwsc5Ue4doiLA1OpiIyxzP1gzXAzNcfYSniUnKKoqI=2FD01AC0"
ValidProjectCategories = ["'Customer Delivery Projects'"]

In [348]:
# Functions
import base64
import pandas as pd
import re 
import requests
from functools import reduce

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        

        response = requests.post(Host + url, headers = headers, json = defaultContents)
        return response.json()
    return fnAPI(ApiCall)  

def fnGetIssueTypeFields(IssueTypes):
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = fnExpandColumn(df, "projects", None)
    df = df[["issuetypes"]]
    df = df.explode("issuetypes")
    df = fnExpandColumn(df, "issuetypes", None)
    df = df[["fields"]]
    df = fnExpandColumn(df, "fields", None)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    df = pd.DataFrame(values)
    df = df.sort_values("fieldId")
    return df 

def fnGetValidProjectKeys():
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    df = fnAPI(ApiCall)
    df = fnExpandColumn(df, "projectCategory")
    df = df.loc[df['projectCategory name'].isin(ValidProjectCategories) | ("'" + df['projectCategory name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnSentenceCase(s):
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]).replace("0", "")

def fnAPI(webRequestDelegate, startAt = 0): 
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df
    
def fnExpandColumn(df:pd.DataFrame, colName:str, prefix:str = "Prefix "):
    if prefix == "Prefix ":
        prefix = colName + " "
    if prefix == None: 
        df = pd.concat([df.drop(colName, axis=1), df[colName].apply(lambda x: pd.Series(x))], axis=1)
    else:
        df = pd.concat([df.drop(colName, axis=1), df[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))], axis=1)
    return df

In [349]:
# Issues Epics

from IPython.display import display
import pandas as pd

ExpectedIssueTypes = [ "Epic" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") ORDER BY updatedDate DESC" 

fields = fnGetIssueTypeFields(ExpectedIssueTypes)

df = fnSearch(JQL, fields["fieldId"].values)

df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
df = df.explode("issues")
df = fnExpandColumn(df, "issues", None)
df = df.drop(["expand", "self"], axis=1)
df = fnExpandColumn(df, "fields", None)
df = df.rename( columns=dict( zip ( fields.fieldId, fields.name )) )
df = df.convert_dtypes().infer_objects()
df["id"] = df["id"].astype('Int64')

display(df.dtypes)
display(df)

id                                   Int64
key                         string[python]
Issue Type                          object
Components                          object
Description                 string[python]
Project                             object
Fix versions                        object
Organizations                       object
Resolution                          object
Approvers                           object
Time tracking                       object
Task progress                       object
Tempo Customer                      object
Min. Estimate                       object
Max. Estimate                       object
Sprint                              object
Epic Link                           object
Attachment                          object
Epic Name                   string[python]
Baseline start date                 object
Resolution Date             string[python]
Program Increment                   object
Baseline end date                   object
Risk probab

,id,key,Issue Type,Components,Description,Project,Fix versions,Organizations,Resolution,Approvers,...,Request participants,Environment,Account,Start date,Affects versions,Due date,Linked Issues,Assignee,Updated,Status
0,48690,HYP-23,{'self': 'https://strategenics.atlassian.net/r...,[],Manufacturers and industrial organisations mus...,{'self': 'https://strategenics.atlassian.net/r...,[],None,None,None,...,[],None,None,<NA>,[],<NA>,[],None,2023-07-20T17:45:41.473+1000,{'self': 'https://strategenics.atlassian.net/r...
0,47421,HYP-21,{'self': 'https://strategenics.atlassian.net/r...,[],<NA>,{'self': 'https://strategenics.atlassian.net/r...,[],None,None,None,...,[],None,None,<NA>,[],<NA>,[],None,2023-07-20T17:45:41.063+1000,{'self': 'https://strategenics.atlassian.net/r...
1,47280,HYP-5,{'self': 'https://strategenics.atlassian.net/r...,[],<NA>,{'self': 'https://strategenics.atlassian.net/r...,[],None,None,None,...,[],None,None,2022-07-24,[],<NA>,[],None,2023-07-20T17:45:40.458+1000,{'self': 'https://strategenics.atlassian.net/r...
1,44545,TRC-334,{'self': 'https://strategenics.atlassian.net/r...,[],<NA>,{'self': 'https://strategenics.atlassian.net/r...,[],None,None,None,...,[],None,"{'id': 8, 'value': 'PMIS'}",2021-02-17,[],<NA>,[],None,2023-07-20T17:39:57.302+1000,{'self': 'https://strategenics.atlassian.net/r...
2,45390,TRC-338,{'self': 'https://strategenics.atlassian.net/r...,[],<NA>,{'self': 'https://strategenics.atlassian.net/r...,[],None,None,None,...,[],None,"{'id': 8, 'value': 'PMIS'}",2021-10-19,[],<NA>,[],None,2023-07-20T17:39:51.130+1000,{'self': 'https://strategenics.atlassian.net/r...
2,44004,TRC-1,{'self': 'https://strategenics.atlassian.net/r...,[],<NA>,{'self': 'https://strategenics.atlassian.net/r...,[],None,None,None,...,[],None,"{'id': 8, 'value': 'PMIS'}",2021-02-17,[],2021-03-19,[],{'self': 'https://strategenics.atlassian.net/r...,2023-07-20T17:39:49.702+1000,{'self': 'https://strategenics.atlassian.net/r...
3,44524,TRC-330,{'self': 'https://strategenics.atlassian.net/r...,[],<NA>,{'self': 'https://strategenics.atlassian.net/r...,[],None,None,None,...,[],None,"{'id': 8, 'value': 'PMIS'}",2021-04-07,[],<NA>,[],None,2023-07-20T17:39:49.067+1000,{'self': 'https://strategenics.atlassian.net/r...
3,44083,TRC-38,{'self': 'https://strategenics.atlassian.net/r...,[],<NA>,{'self': 'https://strategenics.atlassian.net/r...,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,...,[],None,"{'id': 8, 'value': 'PMIS'}",2021-02-17,[],2021-06-30,[],None,2023-07-20T17:39:48.807+1000,{'self': 'https://strategenics.atlassian.net/r...
